In [ ]:
include("../src/HPFEM.jl")

In [ ]:
setprecision(256)
TF = BigFloat
nel = 100
nnodes = nel + 1
idir = [1,nnodes]
M = 30
Q = M+2
b = HPFEM.ModalC01d(M, TF)
quad = HPFEM.QuadType(Q, HPFEM.GLJ, TF)
bas = HPFEM.Basis1d(b,quad, TF)
#bas = HPFEM.SEM1d(M, TF)
lmap = HPFEM.locmap(bas)
dof = HPFEM.DofMap1d(lmap, nnodes, idir);


In [ ]:
π₂ = BigFloat(π)

In [ ]:
k = 1
uexact(x) = sin(2π₂*k*x)
rhsfun(x) = sin(2π₂*k*x) * (one(TF) + (2*one(TF)π₂*k)^2)


In [ ]:
a = parse(TF, "1.3")
b = parse(TF, "6.7")
nodes = [TF(x) for x in linspace(a, b, nnodes)];

In [ ]:
elems = [HPFEM.Element1d(e, nodes[e], nodes[e+1], bas) for e = 1:nel];

In [ ]:
solver = HPFEM.CholeskySC(dof, HPFEM.BBMatrix1d, TF);

In [ ]:
for e = 1:nel
    Ae = zeros(TF, M, M)
    HPFEM.add_stiff_matrix!(bas, elems[e], Ae)
    HPFEM.add_mass_matrix!(bas, elems[e], Ae)
    HPFEM.add_local_matrix(solver, e, Ae)
end


In [ ]:
Fe = zeros(TF, HPFEM.nmodes(lmap), nel)

for e = 1:nel
    fe = rhsfun(elems[e].x)
    HPFEM.add_rhs!(bas, elems[e], fe, view(Fe, :, e))
end

# Apply Dirichilet BCs:
bnd = HPFEM.bndidx(lmap)
Fe[bnd[1],1] = uexact(a);
Fe[bnd[2],nel] = uexact(b);

In [ ]:
HPFEM.solve!(solver, Fe);

In [ ]:
nξ = 101
ξ = collect(linspace(TF(-1),TF(1),nξ));
ϕ = zeros(TF, nξ, M)
for i = 1:M
    ϕ[:,i] = HPFEM.basis1d(bas, ξ, i)
end

Ue = ϕ * Fe;

In [ ]:
using PyPlot
maxerr = zero(TF)
for e = 1:nel
    el = elems[e]
    x = (one(TF)-ξ)*el.a/2 + (one(TF)+ξ)*el.b/2 
    uu = uexact(x)
    err = maxabs(uu-Ue[:,e])
    if err > maxerr maxerr = err end
        
    plot(x, Ue[:,e], "r", x, uu, "b")
end
maxerr
